In [2]:
import pandas as pd

In [3]:
from datetime import datetime, timedelta

In [35]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np

In [4]:
auctions = pd.read_csv("auctions.csv")

In [5]:
events = pd.read_csv("events.csv")

/home/martinrosas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
installs = pd.read_csv("installs.csv")

/home/martinrosas/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
auctions.head()

,date,device_id,ref_type_id,source_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0


In [8]:
#ME QUEDO solo con los 3 primeros dias para uqe la distancia maxima sea de 3 dias
#arranca el 2019-04-18 00:00:00 asi que el date limite es 2019-04-20 23:59:59
limit_date_train = datetime(2019, 4, 20, 23, 59, 59, 999)
#uso los siguientes 3 dias para testear
limit_date_test_begin = datetime(2019, 4, 21, 0, 0, 0, 0)
limit_date_test_end = datetime(2019, 4, 23, 23, 59, 59, 999)

In [9]:
auctions["date"] = pd.to_datetime(auctions["date"])
auctions_train = auctions[auctions["date"] < limit_date_train]
auctions_test = auctions[(auctions["date"] > limit_date_test_begin) & (auctions["date"] < limit_date_test_end)]

In [10]:
#Agrupo por device_id, quedan los dates de apariciones en una lista
deviceIdDates_train = auctions_train.groupby(["device_id"])["date"].apply(list).to_frame().reset_index()

In [11]:
deviceIdDates_test = auctions_test.groupby(["device_id"])["date"].apply(list).to_frame().reset_index()

In [12]:
#Ordeno las fechas
deviceIdDates_train["date"] = deviceIdDates_train["date"].map(lambda x: sorted(x))

In [13]:
deviceIdDates_test["date"] = deviceIdDates_test["date"].map(lambda x: sorted(x))

In [14]:
deviceIdDates_train.head()

,device_id,date
0,41863526108385,"[2019-04-19 19:40:28.465866, 2019-04-20 02:52:..."
1,135153013040192,"[2019-04-20 04:10:54.009137, 2019-04-20 04:18:..."
2,161514654074162,"[2019-04-18 02:52:46.357746, 2019-04-18 02:53:..."
3,181891380775191,[2019-04-20 23:19:25.420614]
4,186034136943920,"[2019-04-18 16:42:46.331894, 2019-04-18 19:23:..."


In [15]:
#Función que resta una fecha de aparición con la que le sigue inmediatamente
#limito a 500 el tamanio para que no itere tantas veces. sino itera muchas veces para los ids que aparecen
#muchas veces. hay muchos que aparecen miles de veces
def nextAppearanceTime( dateList ):
    distancias = []
    longitud_actual = len(dateList) if len(dateList)<500 else 500
    if(longitud_actual > 1):
        for x in range(longitud_actual):
            if(x + 1 < longitud_actual):
                distancias.append((pd.to_datetime(dateList[x + 1]) - pd.to_datetime(dateList[x])).total_seconds())
        
    return distancias

In [16]:
deviceIdDates_train["datesTimeDeltas"] = deviceIdDates_train["date"].map(lambda x: nextAppearanceTime(x))

In [17]:
deviceIdDates_test["datesTimeDeltas"] = deviceIdDates_test["date"].map(lambda x: nextAppearanceTime(x))

In [18]:
#ahora hay una fila por cada device id.
#date es la lista de veces en las cuales aparecio
#y datestimedeltas es el tiempo promedio que tardo en volver a aparecer, y el valor a predecir en el futuro
deviceIdDates_train.head()

,device_id,date,datesTimeDeltas
0,41863526108385,"[2019-04-19 19:40:28.465866, 2019-04-20 02:52:...","[25918.427014, 395.61635, 419.166558, 175.7123..."
1,135153013040192,"[2019-04-20 04:10:54.009137, 2019-04-20 04:18:...","[480.352713, 27.220255, 1.312519, 80.049594, 6..."
2,161514654074162,"[2019-04-18 02:52:46.357746, 2019-04-18 02:53:...","[28.74402, 25.982516, 18.239779, 2.851261, 47...."
3,181891380775191,[2019-04-20 23:19:25.420614],[]
4,186034136943920,"[2019-04-18 16:42:46.331894, 2019-04-18 19:23:...","[9655.874138, 114849.7393, 22.040434, 17.51386..."


In [20]:
#hago que dates time deltas sea un numero en milisegundos
#si es una lista vacia(el id aparecio solamente en 1 auction) reemplazo por el tiempo en segundos equivalente 
# a 3 dias, ya que seria el maximo posible

In [19]:
three_days_seconds = timedelta(3).total_seconds()

In [21]:
deviceIdDates_train["datesTimeDeltas"] = deviceIdDates_train["datesTimeDeltas"].map(lambda x: three_days_seconds if len(x) == 0 else x[0])

In [22]:
deviceIdDates_test["datesTimeDeltas"] = deviceIdDates_test["datesTimeDeltas"].map(lambda x: three_days_seconds if len(x) == 0 else x[0])

In [23]:
#entreno asi nomas un modeo para despues poder usar como ejemplo

In [24]:
#creo un feature que diga si su primer aparecion fue dia de semana o no

In [25]:
deviceIdDates_train["first_appearence_is_weekend"] = deviceIdDates_train.apply(lambda row: row["date"][0].weekday()>4 , axis=1)

In [26]:
deviceIdDates_test["first_appearence_is_weekend"] = deviceIdDates_test.apply(lambda row: row["date"][0].weekday()>4 , axis=1)

In [27]:
#x train va a tener los features ( en este caso solo uno)
X_train = deviceIdDates_train[["first_appearence_is_weekend"]]
X_test = deviceIdDates_test[["first_appearence_is_weekend"]]
#y train es el valor a predecir
y_train = deviceIdDates_train["datesTimeDeltas"]
y_test = deviceIdDates_test["datesTimeDeltas"]

In [28]:
#COMO LO QUE SE PREDICE ES TIEMPO USO UN REGRESOR
#creo el regresor
clf = RandomForestRegressor()
#entreno el modeo
clf.fit(X_train, y_train)  


/home/martinrosas/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [30]:
#esta es la prediccion para cada registro  X test
pred = clf.predict(X_test)
pred

array([46420.10212375, 46420.10212375, 88453.41434792, ...,
       88453.41434792, 88453.41434792, 88453.41434792])

In [25]:
#buscando features

In [32]:
#en este df voy a ir agregando los nuevos features
df_train = deviceIdDates_train[["device_id", "first_appearence_is_weekend", "datesTimeDeltas"]]
df_test = deviceIdDates_test[["device_id", "first_appearence_is_weekend", "datesTimeDeltas"]]

df_train.head()

,device_id,first_appearence_is_weekend,datesTimeDeltas
0,41863526108385,False,25918.427014
1,135153013040192,True,480.352713
2,161514654074162,False,28.744020
3,181891380775191,True,259200.000000
4,186034136943920,False,9655.874138


In [34]:
#creo feature -> para cada id cual es la app mas usada por cada dispositivo a partir del dataframe EVENTS

In [33]:
most_used_app = events.groupby("ref_hash")["application_id"].agg(lambda x: x.mode()).rename_axis("device_id").to_frame().reset_index()
most_used_app.head()

,device_id,application_id
0,40621409780134,77
1,41863526108385,210
2,69039685746313,226
3,90072729247980,210
4,161514654074162,65


In [36]:
#arreglo que algunos casos la app id mas usada es una lista. porque la funcion .mode() devolvio mas de un elemento
most_used_app["application_id"] = most_used_app["application_id"].apply(lambda x: x[0] if type(x) == np.ndarray else x)

In [31]:
#lo agrego al df actual

In [37]:
df_train = df_train.merge(most_used_app, on="device_id", how="left")

In [38]:
df_test = df_test.merge(most_used_app, on="device_id", how="left")

In [39]:
#renombro para dejar mas claro que el nombre del nuevo feature
df_train = df_train.rename({"application_id": "most_used_app"}, axis="columns")
df_test = df_test.rename({"application_id": "most_used_app"}, axis="columns")

In [40]:
#reemplazo los nan de most used app por un nuemero no existente
df_train["most_used_app"] = df_train["most_used_app"].fillna(-1)
df_test["most_used_app"] = df_test["most_used_app"].fillna(-1)

In [41]:
df_train.head()

,device_id,first_appearence_is_weekend,datesTimeDeltas,most_used_app
0,41863526108385,False,25918.427014,210.0
1,135153013040192,True,480.352713,-1.0
2,161514654074162,False,28.744020,65.0
3,181891380775191,True,259200.000000,-1.0
4,186034136943920,False,9655.874138,210.0


In [42]:
#creo feature para cada ID el sistema operativo a partir de dataframe Installs

In [43]:
def is_android_or_macos(row):
    if row["user_agent"] is np.nan:
        return None
    agent = row["user_agent"].lower()
    if "android" in agent:
        return "android"
    elif "darwin" in agent:
        return "mac os"
    else:
        return "other"

In [44]:
installs["os"] = installs.apply(is_android_or_macos, axis=1)

In [45]:
os = installs[["ref_hash", "os"]].dropna()

In [46]:
#saco ids duplicados
os = os.drop_duplicates(subset="ref_hash", keep='first').rename({"ref_hash": "device_id"},axis="columns")

In [52]:
#lo agrego a DF, siempre left join
df_train = df_train.merge(os,how="left", on="device_id")
df_test = df_test.merge(os,how="left", on="device_id")
df_train.head()

,device_id,first_appearence_is_weekend,datesTimeDeltas,most_used_app,os_x,os_y,os_x,os_y,os
0,41863526108385,False,25918.427014,210.0,android,android,android,android,android
1,135153013040192,True,480.352713,-1.0,NaN,NaN,NaN,NaN,NaN
2,161514654074162,False,28.744020,65.0,android,android,android,android,android
3,181891380775191,True,259200.000000,-1.0,NaN,NaN,NaN,NaN,NaN
4,186034136943920,False,9655.874138,210.0,NaN,NaN,NaN,NaN,NaN


In [42]:
#como hay muchos que no tienen info de sistema operativo, relleno los NA con "other"

In [53]:
df_train["os"] = df_train["os"].fillna("other")
df_test["os"] = df_test["os"].fillna("other")

In [54]:
#PRUEBO OTRA VEZ con random forest
#antes tengo que convertir a numeros los campos que son string (en este caso solo OS)
#USO LABEL ENCODER
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [55]:
#encodeo columna os
le.fit(df_train["os"])
df_train["os"] = le.transform(df_train["os"])
df_test["os"] = le.transform(df_test["os"])

In [56]:
#ahora en vez de string tengo sus codigos
df_train["os"].head()

0    0
1    2
2    0
3    2
4    2
Name: os, dtype: int64

In [57]:

#CREO X_TRAIN Y Y_TRAIN
X_train = df_train[['first_appearence_is_weekend',
       'most_used_app', 'os']]
y_train = df_train['datesTimeDeltas']
X_test = df_test[['first_appearence_is_weekend',
       'most_used_app', 'os']]
y_test = df_test['datesTimeDeltas']

In [60]:
#entreno nuevo arbol asi nomas

In [64]:
clf = RandomForestRegressor()
#entreno el modeo
clf.fit(X_train, y_train)  
y_pred = clf.predict(X_test)

/home/martinrosas/.local/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [65]:
#en y_pred tengo las predicciones y en y_test tengo los valores de verdad que queria predecir
#mido el error

print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))

Mean squared error: 10203212611.79


In [66]:
#ahora busco mejores features para bajar el error